In [ ]:
%load_ext dotenv
%dotenv .env

In [ ]:
from redbox.app import Redbox
from redbox.models.settings import Settings, get_settings
from redbox.models.chain import RedboxQuery, RedboxState, AISettings, ChatLLMBackend
from langfuse.callback import CallbackHandler
from uuid import uuid4

In [ ]:
def get_state(user_uuid, prompts, documents, ai_setting):
    q = RedboxQuery(
        question=f"@gadget {prompts[-1]}",
        s3_keys=documents,
        user_uuid=user_uuid,
        chat_history=prompts[:-1],
        ai_settings=ai_setting,
        permitted_s3_keys=documents,
    )

    return RedboxState(
        request=q,
    )

In [ ]:
def run_app(app, state) -> RedboxState:
    langfuse_handler = CallbackHandler()
    return app.graph.invoke(state, config={"callbacks": [langfuse_handler]})

In [ ]:
env = get_settings()
ai_setting = AISettings(chat_backend=ChatLLMBackend(name="anthropic.claude-3-sonnet-20240229-v1:0", provider="bedrock"))
app = Redbox(debug=True, env=env)
x = get_state(uuid4(), prompts = ['What is the capital of France?'], documents = [], ai_setting = ai_setting)
run_app(app, x)